# This is a sample Jupyter Notebook

In [1]:
import pandas as pd

In [2]:
import os
raw_data_path = f'{os.getcwd()}/raw_data'
cleaned_data_path = f'{os.getcwd()}/cleaned_data'
os.makedirs(cleaned_data_path, exist_ok=True)

In [3]:
# -------------------------------
# Column name normalisation
# -------------------------------
def normalise_column_names(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns.str.lower()
                  .str.strip()
                  .str.replace(" ", "_")
                  .str.replace(r"[^a-z0-9_]", "", regex=True)
                  .str.replace(r"_+", "_", regex=True)
    )
    return df


# -------------------------------
# Standardise missing values
# -------------------------------
MISSING_STRINGS = {
    "na", "n/a", "n\\a", "nan", "<na>", "none", "null", "nil",
    "", " ", "  ", "-", "--", "N/A", "NaN", "NA", "NULL", "None"
}

def standardise_missing(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df = df.replace(MISSING_STRINGS, pd.NA)
    df = df.replace(r"^\s+$", pd.NA, regex=True)
    return df


# -------------------------------
# Drop irrelevant columns
# -------------------------------
def drop_columns(df: pd.DataFrame, col_list) -> pd.DataFrame:
    return df.drop(columns=col_list, errors="ignore")


# -------------------------------
# Clean categorical string columns
# -------------------------------
def clean_string_cols(df: pd.DataFrame, cols) -> None:
    for col in cols:
        df[col] = (
            df[col]
            .astype("string")
            .str.strip()
            .str.lower()
            .str.replace(r"\s+", "_", regex=True)
            .str.replace(r"[^a-z0-9_]", "", regex=True)
        )


# -------------------------------
# Extract postcode + district
# -------------------------------
def add_postcode_district(df: pd.DataFrame, address_col: str) -> pd.DataFrame:
    df = df.copy()

    df["postcode"] = df[address_col].str.extract(
        r"([A-Z]{1,2}\d{1,2}[A-Z]?\s*\d[A-Z]{2})",
        expand=False
    )

    df["postcode"] = (
        df["postcode"]
        .str.replace(r"\s+", "", regex=True)
        .str.upper()
    )

    df["postcode_district"] = df["postcode"].str[:-3]

    return df.drop(columns=[address_col, "postcode"])

In [6]:
# -------------------------------
# 1. Load
# -------------------------------
camden_pa = (
    pd.read_csv(
        f"{raw_data_path}/planning_application_camden.csv",
        index_col=0,
        dtype=str,
        low_memory=False,
        parse_dates=True,
        date_format="%d/%m/%Y"
    )
    .convert_dtypes()
)

# -------------------------------
# 2. Normalise column names
# -------------------------------
camden_pa = normalise_column_names(camden_pa)

# -------------------------------
# 3. Standardise missing
# -------------------------------
camden_pa = standardise_missing(camden_pa)

# -------------------------------
# 4. Drop duplicates and irrelevant cols
# -------------------------------
camden_pa = camden_pa.drop_duplicates()

DROP_IRRELEVANT = [
    "application_number", "case_officer", "case_officer_team", "applicant_name",
    "responsibility_type", "comment", "full_application", "spatial_accuracy",
    "last_uploaded", "socrata_id", "organisation_uri", "earliest_decision_date",
    "easting", "northing", "location", "registered_in_last_7_working_days",
    "registered_in_last_28_working_days", "valid_from_date", "registered_date",
    "decision_date", "system_status_change_date", "longitude", "latitude"
]
DROP_NOT_KNOWN_BEFORE_APPLICATION = [
    "decision_level", "system_status"
]

camden_pa = drop_columns(
    camden_pa,
    DROP_IRRELEVANT + DROP_NOT_KNOWN_BEFORE_APPLICATION
)

# -------------------------------
# 5. Core fields required
# -------------------------------
camden_pa = camden_pa.dropna(subset=["decision_type", "ward"])

# -------------------------------
# 6. Standardise string columns
# -------------------------------
STRING_COLS = [
    "decision_type", "conservation_areas",
    "neighbourhood_areas", "application_type", "ward"
]
clean_string_cols(camden_pa, STRING_COLS)

# -------------------------------
# 7. Decision outcome mapping
# -------------------------------
POSITIVE_DECISIONS = [
    "granted", "no_objection", "no_objection_to_works_to_tree(s)_in_ca",
    "no_objection_to_emergency_works_(ca)", "approve_works_(tpo)",
    "approve_emergency_works_(tpo)", "granted_subject_to_a_section_106_legal_agreement",
    "granted_and_enforcement_action_to_be_taken",
    "granted_and_warning_of_enforcement_action", "grant_prior_approval",
    "prior_approval_required_-_approval_given", "prior_approval_not_required",
    "granted_prior_approval_subject_to_section_106_legal_agreement",
    "approval", "approve_heritage_partnership_agreement",
    "agree_s106_deed_of_variation", "grant_certificate,_refuse_application",
    "deemed_consent_-_no_objection"
]

camden_pa = camden_pa[
    camden_pa["decision_type"].str.strip().str.lower() != "withdrawn_decision"
]

camden_pa["outcome"] = camden_pa["decision_type"].isin(POSITIVE_DECISIONS).astype(int)
camden_pa = camden_pa.drop(columns=["decision_type"])

# -------------------------------
# 8. Conservation and neighbourhood flags
# -------------------------------
camden_pa["in_conservation_area"] = camden_pa["conservation_areas"].notna().astype(int)
camden_pa["conservation_areas"] = camden_pa["conservation_areas"].fillna("none")

camden_pa["in_neighbourhood_area"] = camden_pa["neighbourhood_areas"].notna().astype(int)
camden_pa["neighbourhood_areas"] = camden_pa["neighbourhood_areas"].fillna("none")

# -------------------------------
# 9. Postcode extraction
# -------------------------------
camden_pa = add_postcode_district(camden_pa, "development_address")


# -------------------------------
# 10. Clean ward categories
# -------------------------------
def clean_ward(series):
    s = series.str.replace(r"\(pre_may_2022\)", "", regex=True)
    s = s.str.split(";").str[0]
    s = s.str.replace("_&_", "_and_", regex=False)
    s = s.str.replace("&", "and", regex=False)

    s = (
        s.str.lower()
         .str.replace(" ", "_")
         .str.replace("[^a-z0-9_]", "", regex=True)
         .str.strip("_")
    )

    counts = s.value_counts()
    rare = counts[counts < 20].index
    return s.replace(rare, "other")

camden_pa["ward"] = clean_ward(camden_pa["ward"])

# -------------------------------
# 11. Save
# -------------------------------
camden_pa.to_parquet(
    f"{cleaned_data_path}/cleaned_camden_pa.parquet",
    engine="fastparquet"
)

camden_pa

,development_description,ward,conservation_areas,neighbourhood_areas,application_type,outcome,in_conservation_area,in_neighbourhood_area,postcode_district
ID,,,,,,,,,
"244,936",FRONT GARDEN: 1 x Horse Chestnut - Repollard,swiss_cottage_pre_may_2022,none,none,notification_of_intended_works_to_trees_in_a_c...,0,0,0,NW6
"275,411",Amendments including the introduction of two p...,st_pancras__somers_town_pre_may_2022,regents_canal,none,approval_of_reserved_matters,1,1,0,<NA>
"313,221",Display of non-illuminated projecting sign.,holborn__covent_garden_pre_may_2022,seven_dials_covent_garden,none,advertisement_consent,1,1,0,WC2B
"319,324",Observations to the City of Westminster for th...,kentish_town_pre_may_2022,none,none,request_for_observations_to_adjoining_borough,1,0,0,NW6
"324,126",Conversion of 2 x self contained flats into a ...,belsize_pre_may_2022,belsize_park,none,full_planning_permission,1,1,0,NW3
...,...,...,...,...,...,...,...,...,...
"644,433",Replacement of the existing soffit and lightin...,bloomsbury,none,none,full_planning_permission,1,0,0,W1T
"215,454",Installation of new shop front to existing hot...,bloomsbury_pre_may_2022,charlotte_street,none,full_planning_permission,1,1,0,W1T
"641,189",1 x car painted graphic and 1 x cyclist painte...,bloomsbury,bloomsbury,none,advertisement_consent,1,1,0,WC1A


In [7]:
# -------------------------------
# 1. Load
# -------------------------------
appeals = (
    pd.read_csv(
        f"{raw_data_path}/planning_application_appeal_uk.csv",
        index_col=0,
        low_memory=False,
        dtype=str
    )
    .convert_dtypes()
)

# -------------------------------
# 2. Normalise columns
# -------------------------------
appeals = normalise_column_names(appeals)

# -------------------------------
# 3. Standardise missing
# -------------------------------
appeals = standardise_missing(appeals)

# -------------------------------
# 4. Drop duplicates and rows missing required dates
# -------------------------------
appeals = appeals.drop_duplicates()
appeals = appeals.dropna(subset=["decision", "received_date", "valid_date", "start_date"])

# -------------------------------
# 5. Drop irrelevant columns
# -------------------------------
DROP_IRRELEVANT = [
    "ons_lpa_code", "appellant", "agent", "link_status", "lead_case",
    "lpa_application_reference", "date_recovered",
    "date_not_recovered_or_derecovered", "call_in_date",
    "costs_applied_for_indicator", "inspector_name",
    "enforcement_grounds_count", "enforcement_grounds",
    "lpa_decision_date", "decision_date", "bespoke"
]
DROP_NOT_KNOWN_BEFORE_APPEAL = [
    "jurisdiction", "redetermined",
    "received_date", "valid_date", "start_date"
]

appeals = drop_columns(
    appeals,
    DROP_IRRELEVANT + DROP_NOT_KNOWN_BEFORE_APPEAL
)

# -------------------------------
# 6. Remove unwanted decisions
# -------------------------------
DECISIONS_TO_DROP = {
    "split_decision", "unknown", "no_decision_code_associated"
}

appeals = appeals[~appeals["decision"].str.lower().isin(DECISIONS_TO_DROP)]

# -------------------------------
# 7. Standardise string columns
# -------------------------------
STRING_COLS = [
    "type_of_casework", "lpa_name", "decision", "procedure",
    "development_type", "reason_for_the_appeal", "type_detail",
    "site_green_belt"
]
clean_string_cols(appeals, STRING_COLS)

# -------------------------------
# 8. Outcome mapping
# -------------------------------
positive_decisions = {
    "allowed", "allowed_with_conditions", "planning_permission_granted",
    "allowed_in_part", "notice_quashed"
}

appeals["outcome"] = appeals["decision"].isin(positive_decisions).astype(int)
appeals = appeals.drop(columns=["decision"])

# -------------------------------
# 9. Binary columns
# -------------------------------
appeals["site_green_belt"] = appeals["site_green_belt"].map({"yes": 1, "no": 0}).fillna(0).astype(int)

BINARY_COLS = [
    "number_of_residences", "agricultural_holding",
    "development_affect_setting_of_listed_building",
    "historic_building_grant_made", "in_ca_relates_to_ca",
    "is_flooding_an_issue", "is_the_site_within_an_aonb",
    "is_site_within_an_sssi"
]
appeals[BINARY_COLS] = appeals[BINARY_COLS].fillna(0).astype(int)

# -------------------------------
# 10. Numeric columns
# -------------------------------
for col in ["area_of_site_in_hectares", "floor_space_in_square_metres"]:
    appeals[col] = pd.to_numeric(appeals[col], errors="coerce")
    appeals[col] = appeals[col].mask(appeals[col] < 0, pd.NA)

# -------------------------------
# 11. Postcode extraction
# -------------------------------
appeals = add_postcode_district(appeals, "site_address")

# -------------------------------
# 12. Appeal type reason
# -------------------------------
appeals["appeal_type_reason"] = (
    appeals["appeal_type_reason"]
    .astype("string")
    .str.strip()
    .str.lower()
    .str.replace(r"^\d+\.\s*", "", regex=True)
    .fillna("none")
)

appeals["reason_for_the_appeal"] = appeals["reason_for_the_appeal"].fillna("none")

# -------------------------------
# 13. Save
# -------------------------------
appeals.to_parquet(
    f"{cleaned_data_path}/cleaned_appeals.parquet",
    engine="fastparquet"
)

appeals

,type_of_casework,lpa_name,procedure,development_type,appeal_type_reason,reason_for_the_appeal,type_detail,area_of_site_in_hectares,floor_space_in_square_metres,number_of_residences,site_green_belt,agricultural_holding,development_affect_setting_of_listed_building,historic_building_grant_made,in_ca_relates_to_ca,is_flooding_an_issue,is_the_site_within_an_aonb,is_site_within_an_sssi,outcome,postcode_district
Case Number,,,,,,,,,,,,,,,,,,,,
3226323,enforcement_notice,dacorum_borough_council,written_representations,other_minor_developments,lpa's service of an enforcement notice,none,unknown,<NA>,<NA>,0,1,0,0,0,0,0,0,0,0,LU1
3240681,planning_appeal,woking_borough_council,written_representations,major_dwellings,refused planning permission,refusal,planning_appeals__outline,0,<NA>,38,0,0,0,0,0,0,0,0,0,GU21
3241431,enforcement_notice,rossendale_borough_council,written_representations,change_of_use,lpa's service of an enforcement notice,none,unknown,<NA>,<NA>,0,0,0,0,0,0,0,0,0,0,<NA>
3241496,lawful_development_certificate,southampton_city_council,written_representations,change_of_use,the decision of the lpa refusing/refusing in p...,refusal,unknown,<NA>,<NA>,0,0,0,0,0,0,0,0,0,1,SO18
3242222,planning_appeal,ribble_valley_borough_council,written_representations,minor_dwellings,refused planning permission,refusal,planning_appeals__full,0,<NA>,1,0,0,0,0,0,0,0,0,0,BB7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6000233,householder_has,royal_borough_of_greenwich,written_representations,householder_developments,none,refused,householder,0,222,0,0,0,1,0,0,0,0,0,0,SE2
6000339,householder_has,royal_borough_of_greenwich,written_representations,householder_developments,none,refused,householder,0,137,0,0,0,1,0,0,0,0,0,1,SE7
HH-2154,high_hedges,doncaster_metropolitan_borough_council,written_representations,no_development_type_associated,none,none,no_application_type_assocaited,<NA>,<NA>,0,0,0,0,0,0,0,0,0,0,<NA>
